In [1]:
%matplotlib inline

import os
import sys
import math
import pandas as pd
import multiprocessing as mp
import matplotlib.pyplot as plt
from functools import partial

module_path = os.path.abspath(os.path.join('..'))
src_path = '{}/src'.format(module_path)
if module_path not in sys.path:
    sys.path.append(src_path)
    
from preprocessing import select
from consts import dataset, model_cols

plt.rcParams.update({'figure.max_open_warning': 0})

# Preparando a base de dados.
## Selecionando os atributos identificados como relevantes.

- Primeiro vamos definir as colunas a serem utilizadas;
- Depois, vamos utilizar
pequenos chunks para processar a base;
- Por fim, vamos salvar o arquivo
processado;
- No caso de já existir um arquivo CSV, ele será carregado.

In [2]:
save_path = '{}/data/processed/dataset.csv'.format(module_path)
load_path ='{}/data/raw/pessoas.txt'.format(module_path)

try:
    df_dataset = pd.read_csv(save_path)
except FileNotFoundError:
    # Preparando para processar o arquivo original.
    chunksize = 500
    pool = mp.Pool(mp.cpu_count())
    reader = pd.read_fwf(
        load_path,
        widths=dataset['col_widths'],
        names=dataset['col_names'],
        chunksize=chunksize
    )

    # Processamento do arquivo original.
    partial_select = partial(select, cols=model_cols)
    df_dataset = pd.concat(pool.map(partial_select, reader))
    df_dataset = df_dataset.replace('^\.$', '', regex=True)

    # Salvando arquivo processado.
    df_dataset.to_csv(save_path, header=True, index=False)

    # Dimensões da base
    df_dataset.shape

### Selecionando instâncias diagnosticadas com depressão

- Primeiro, será feita uma query buscando as instâncias onde o campo Diagnosticado_Depressao é igual 1;
- Depois, o resultado será salvo em um novo arquivo csv;
- Caso o arquivo já exista, basta ser carregado.

In [3]:
df_dep = df_dataset.query('Diagnosticado_Depressao == 1')
df_dep.shape

(4235, 68)

### Selecionando instâncias não diagnosticadas com depressão

- Primeiro, será feita uma query buscando as instâncias onde o campo Diagnosticado_Depressao é igual 2;
- Depois, o resultado será salvo em um novo arquivo csv;
- Caso o arquivo já exista, basta ser carregado.

In [4]:
df_ndep = df_dataset.query('Diagnosticado_Depressao == 2').sample(df_dep.shape[0])
df_ndep.shape

(4235, 68)

### Unindo as bases

O objetivo é trabalhar com instâncias diagnosticadas ou não com depressão. Para tanto, iremos concatenar as duas bases geradas.

In [5]:
sample_path = '{}/data/processed/sample.csv'.format(module_path)
df_sample = pd.concat([df_dep, df_ndep])
df_sample.to_csv(sample_path, header=True, index=False)
df_sample.shape

(8470, 68)